# Build your first Crew
* Step-by-step tutorial to create a Crew that works to solve complex problems.
    * Research a topic.
    * Analize the findings.
    * Create a comprehensive report about it. 

## This technique can be applied to build other projects like:
* **Content Creation and Edition**: Multi-stage content creation with specialized writers, editors, and fact-checkers.
* **Customer Service**: Complex customer service systems with tiered support agents.
* **Business Analysis**: Autonomous business analysts that gather data, create visualizations, and generate insights.
* **Product Development**: Product development teams that ideate, design, and plan implementation.

## Before starting, you will need to have:
* CrewAI installed.
* Create a .env file with your OpenAI API Key on it.

## Step 1: Create a new CrewAI Project
In terminal:
* `crewai create crew research_crew`
* `cd research_crew`

This will generate a template for our project, with:
* A project directory with the necessary files.
* Configuration files for agents and tasks.
* A basic crew implementation.
* A main script to run the crew.

## Step 2: Explore the Project Structure
```text
research_crew/
├── .gitignore
├── pyproject.toml
├── README.md
├── .env
└── src/
    └── research_crew/
        ├── __init__.py
        ├── main.py
        ├── crew.py
        ├── tools/
        │   ├── custom_tool.py
        │   └── __init__.py
        └── config/
            ├── agents.yaml
            └── tasks.yaml
```

The project structure follows best practices for Python projects and makes it easy to organize your code. 

The separation of configuration files (in YAML) from implementation code (in Python) makes it easy to modify your crew’s behavior without changing the underlying code.

#### Let's break down some of the secondary components of the project structure:

#### `.gitignore`
- Specifies which files and directories Git should ignore (e.g., `__pycache__`, `.env`, `*.pyc`, virtual environment folders).
- Helps avoid committing sensitive or unnecessary files to version control.

#### `.env`
- A file to store **environment variables** (e.g., API keys, tokens, configuration values) used by the application.

#### `pyproject.toml`
- The **central configuration file** for Python projects.
- Defines dependencies, build system, and metadata for publishing (e.g., version, author).

#### `README.md`
- The **documentation hub** of your project.
- Describes what the project does, how to install/run it, and provides usage examples or contributor guidelines.

##### `__init__.py`
- Marks the folder as a Python package.
- Can be used to expose top-level API objects if needed.

## Step 3: Customize the definition of the Agents in `agents.yaml`
We’ll create two agents:
* A researcher who excels at finding and organizing information.
* An analyst who can interpret research findings and create insightful reports.

```yaml
# src/research_crew/config/agents.yaml
researcher:
  role: >
    Senior Research Specialist for {topic}
  goal: >
    Find comprehensive and accurate information about {topic}
    with a focus on recent developments and key insights
  backstory: >
    You are an experienced research specialist with a talent for
    finding relevant information from various sources. You excel at
    organizing information in a clear and structured manner, making
    complex topics accessible to others.
  llm: openai/gpt-4o-mini

analyst:
  role: >
    Data Analyst and Report Writer for {topic}
  goal: >
    Analyze research findings and create a comprehensive, well-structured
    report that presents insights in a clear and engaging way
  backstory: >
    You are a skilled analyst with a background in data interpretation
    and technical writing. You have a talent for identifying patterns
    and extracting meaningful insights from research data, then
    communicating those insights effectively through well-crafted reports.
  llm: openai/gpt-4o-mini
```

## Step 4: Customize the definition of the Tasks in `tasks.yaml`
We’ll create two tasks:
* A research task for gathering comprehensive informationv
* An analysis task for creating an insightful report.

```yaml
# src/research_crew/config/tasks.yaml
research_task:
  description: >
    Conduct thorough research on {topic}. Focus on:
    1. Key concepts and definitions
    2. Historical development and recent trends
    3. Major challenges and opportunities
    4. Notable applications or case studies
    5. Future outlook and potential developments

    Make sure to organize your findings in a structured format with clear sections.
  expected_output: >
    A comprehensive research document with well-organized sections covering
    all the requested aspects of {topic}. Include specific facts, figures,
    and examples where relevant.
  agent: researcher

analysis_task:
  description: >
    Analyze the research findings and create a comprehensive report on {topic}.
    Your report should:
    1. Begin with an executive summary
    2. Include all key information from the research
    3. Provide insightful analysis of trends and patterns
    4. Offer recommendations or future considerations
    5. Be formatted in a professional, easy-to-read style with clear headings
  expected_output: >
    A polished, professional report on {topic} that presents the research
    findings with added analysis and insights. The report should be well-structured
    with an executive summary, main sections, and conclusion.
  agent: analyst
  context:
    - research_task
  output_file: output/report.md
```

**Note the context field in the analysis task** - this is a powerful feature that allows the analyst to access the output of the research task. This creates a workflow where information flows naturally between agents, just as it would in a human team.

## Step 5: Customize the definition of the Crew in `crew.py`

In [ ]:
# src/research_crew/crew.py
from crewai import Agent, Crew, Process, Task
from crewai.project import CrewBase, agent, crew, task
from crewai_tools import SerperDevTool
from crewai.agents.agent_builder.base_agent import BaseAgent
from typing import List

@CrewBase
class ResearchCrew():
    """Research crew for comprehensive topic analysis and reporting"""

    agents: List[BaseAgent]
    tasks: List[Task]

    @agent
    def researcher(self) -> Agent:
        return Agent(
            config=self.agents_config['researcher'], # type: ignore[index]
            verbose=True,
            tools=[SerperDevTool()]
        )

    @agent
    def analyst(self) -> Agent:
        return Agent(
            config=self.agents_config['analyst'], # type: ignore[index]
            verbose=True
        )

    @task
    def research_task(self) -> Task:
        return Task(
            config=self.tasks_config['research_task'] # type: ignore[index]
        )

    @task
    def analysis_task(self) -> Task:
        return Task(
            config=self.tasks_config['analysis_task'], # type: ignore[index]
            output_file='output/report.md'
        )

    @crew
    def crew(self) -> Crew:
        """Creates the research crew"""
        return Crew(
            agents=self.agents,
            tasks=self.tasks,
            process=Process.sequential,
            verbose=True,
        )

In the previous code, we’re:
* Creating the researcher agent and equipping it with the SerperDevTool to search the web.
* Creating the analyst agent.
* Setting up the research and analysis tasks.
* Configuring the crew to run tasks sequentially (the analyst will wait for the researcher to finish).

## The previous code may be difficult to understand for begginers. Let's explain it firts in simple terms and then we will explain it line by line.
This CrewAI code defines a structured AI workflow for researching and analyzing a topic using specialized agents and tasks.

#### Overview

The code sets up a "crew" of AI agents, each assigned specific roles and tasks to collaboratively perform comprehensive topic analysis and reporting.

#### Agents

1. **Researcher Agent**:
   - **Role**: Conducts in-depth research on a given topic.
   - **Tools**: Utilizes the `SerperDevTool` for web searches. 
   - **Configuration**: Settings are loaded from `agents_config['researcher']`.

2. **Analyst Agent**:
   - **Role**: Analyzes the research findings to generate insights.
   - **Tools**: No additional tools specified.
   - **Configuration**: Settings are loaded from `agents_config['analyst']`.

#### Tasks

1. **Research Task**:
   - **Description**: Instructs the Researcher Agent to gather information on the topic.
   - **Configuration**: Details are loaded from `tasks_config['research_task']`.

2. **Analysis Task**:
   - **Description**: Instructs the Analyst Agent to analyze the research data.
   - **Output**: Results are saved to `output/report.md`.
   - **Configuration**: Details are loaded from `tasks_config['analysis_task']`.

#### Crew Assembly

The `crew` method assembles the agents and tasks into a cohesive unit:

- **Agents**: Includes both the Researcher and Analyst Agents.
- **Tasks**: Includes both the Research and Analysis Tasks.
- **Process**: Set to `Process.sequential`, meaning tasks are executed in order. 
- **Verbose**: Enabled for detailed logging during execution.

#### Summary

This code defines a two-agent system where:

1. The **Researcher Agent** uses the `SerperDevTool` to gather information on a topic.
2. The **Analyst Agent** processes this information to generate a detailed report.

The workflow is sequential, ensuring that research is completed before analysis begins.

## Now let's explain the previous code line by line

```python
from crewai import Agent, Crew, Process, Task
```
**Import**: Brings in the main classes (`Agent`, `Crew`, `Process`, `Task`) from the `crewai` library.

---

```python
from crewai.project import CrewBase, agent, crew, task
```
**Import**: Brings in decorators (`CrewBase`, `agent`, `crew`, `task`) used to easily mark methods and classes for CrewAI.

#### Let's review this: what are decorators in Python?

**Decorators** are a special kind of function in Python that **modify or enhance** other functions or classes **without changing their code directly**.

They are written with the **`@` symbol** just above a function or class. Example:

```python
@decorator_name
def my_function():
    pass
```

This is **exactly the pattern you see** in CrewAI code, like:

```python
@agent
def researcher(self) -> Agent:
    ...
```

#### So what does this line mean?

```python
from crewai.project import CrewBase, agent, crew, task
```

You're importing four **decorators** from the `crewai.project` module:

- `@CrewBase`: marks a **class** as a CrewAI project base.
- `@agent`: marks a **method** that defines an agent.
- `@task`: marks a **method** that defines a task.
- `@crew`: marks the method that **assembles** the final Crew object.

#### Why use decorators?

They:
- Help CrewAI **automatically recognize and organize** your agents, tasks, and setup.
- Keep your code clean and declarative.
- Hide complex setup logic behind a single line.

#### Here's a very simple example to help you understand **how decorators work under the hood**.

Example: Create Your Own Decorator

```python
def my_decorator(func):
    def wrapper():
        print("✨ Before the function runs")
        func()
        print("✅ After the function runs")
    return wrapper
```

This is a basic decorator called `my_decorator`. Now let's use it on a function:

```python
@my_decorator
def say_hello():
    print("👋 Hello!")

say_hello()
```

#### What happens when you run this?

Output:
```
✨ Before the function runs
👋 Hello!
✅ After the function runs
```

This is exactly the same as writing:

```python
say_hello = my_decorator(say_hello)
```

#### Summary

A decorator:
- Takes a function as input,
- Wraps it with some additional behavior (before/after/etc),
- Returns the new version of the function.

CrewAI decorators like `@agent`, `@task`, or `@crew` do something similar: they wrap your method or class so CrewAI knows how to use it in a project.

---

```python
from crewai_tools import SerperDevTool
```
**Import**: Imports a tool (`SerperDevTool`) that helps agents perform Google-like online searches.

---

```python
from crewai.agents.agent_builder.base_agent import BaseAgent
```
**Import**: Imports the base type (`BaseAgent`) for creating agent templates.

---

```python
from typing import List
```
**Import**: Allows specifying a list type hint, so Python knows certain variables will be lists.

---

```python
@CrewBase
class ResearchCrew():
```
**Decorator + Class**: Marks the class `ResearchCrew` as a **CrewBase**, meaning it defines a set of agents and tasks.

---

```python
    """Research crew for comprehensive topic analysis and reporting"""
```
**Docstring**: A short description saying this class is about researching and reporting on a topic.

---

```python
    agents: List[BaseAgent]
    tasks: List[Task]
```
**Attributes**:  
- `agents`: A list where all agents (workers) will be stored.  
- `tasks`: A list where all tasks (jobs) will be stored.

---

```python
    @agent
    def researcher(self) -> Agent:
```
**Method (agent decorator)**: Defines a method that returns an **Agent** called **researcher**.

---

```python
return Agent(
    config=self.agents_config['researcher'], # type: ignore[index]
    verbose=True,
    tools=[SerperDevTool()]
)
```

This line is **creating and returning a new `Agent`** object (from the `crewai` library).  

Here's what **each piece** means:

#### `config=self.agents_config['researcher']`
- **Meaning**:  
  The agent needs configuration — things like its name, role, goals, behavior style, etc.
  
- **Where it comes from**:  
  `self.agents_config` is a dictionary that holds settings for all agents.  
  You pick the **'researcher'** config from that dictionary.

#### `# type: ignore[index]`
- **Meaning**:  
  A note to **the Python type checker** (like `mypy`) to **ignore** any type errors about indexing.
  
- **Why it's needed**:  
  Sometimes static type checkers get confused about dynamic dictionaries like `agents_config`, so this avoids unnecessary warnings.

- **It does **NOT** affect the running code.** It's only for tools like `mypy` or editors (e.g., VSCode).

#### `verbose=True`
- **Meaning**:  
  Turn on **detailed logging** when the agent runs.
  
- **Result**:  
  You will **see prints** or **logs** of what the agent is doing — good for debugging and learning.

- **Without `verbose=True`**:  
  The agent would run silently unless an error happened.

#### `tools=[SerperDevTool()]`
- **Meaning**:  
  Give the agent **external tools** it can use during its work.

- **In this case**:  
  `SerperDevTool` is a tool that allows the agent to **search the web** — like using a mini-Google inside the AI.

- **Why important?**  
  Without tools, an agent is "blind" to the outside world.  
  Tools expand the agent's capabilities beyond just reasoning.

- **Example of what the Researcher could do**:
  > "Search the web for the latest news about renewable energy trends."

#### Putting it all together

This code:
- Builds a Researcher Agent,
- Configures its **identity and goal** (`config`),
- **Turns on detailed logging** (`verbose=True`),
- **Gives it a search tool** to look up information (`tools=[SerperDevTool()]`).

Then the method immediately **returns** this new Agent object.


---

```python
    @agent
    def analyst(self) -> Agent:
```
**Method (agent decorator)**: Defines another agent called **analyst**.

---

```python
        return Agent(
            config=self.agents_config['analyst'], # type: ignore[index]
            verbose=True
        )
```
**Create Analyst Agent**:
- Gets settings from `agents_config['analyst']`.
- Also logs actions (`verbose=True`).
- **No extra tools** here—only works with internal capabilities.

---

```python
    @task
    def research_task(self) -> Task:
```
**Method (task decorator)**: Defines a **task** called **research_task**.

---

```python
        return Task(
            config=self.tasks_config['research_task'] # type: ignore[index]
        )
```
**Create Research Task**:
- Pulls task settings from `tasks_config['research_task']`.
- This task tells the Researcher what they should find out.

---

```python
    @task
    def analysis_task(self) -> Task:
```
**Method (task decorator)**: Defines another **task** called **analysis_task**.

---

```python
        return Task(
            config=self.tasks_config['analysis_task'], # type: ignore[index]
            output_file='output/report.md'
        )
```
**Create Analysis Task**:
- Pulls settings from `tasks_config['analysis_task']`.
- Specifies that the result of the analysis will be **saved** to `output/report.md`.

---

```python
    @crew
    def crew(self) -> Crew:
```
**Method (crew decorator)**: Defines how all agents and tasks are organized into one **Crew**.

---

```python
        """Creates the research crew"""
```
**Docstring**: Short description: "Creates the research crew."

---

```python
        return Crew(
            agents=self.agents,
            tasks=self.tasks,
            process=Process.sequential,
            verbose=True,
        )
```
**Create Crew**:
- Combines **agents** and **tasks** into a team.
- `Process.sequential` means: do tasks one after another (not in parallel).
- `verbose=True` turns on detailed logging during the crew's work.

---

#### Final Summary
This file builds a **Research Crew** in CrewAI, where:
- A **Researcher Agent** researches a topic.
- An **Analyst Agent** analyzes the research.
- Tasks are performed one at a time.
- The analysis is saved into a Markdown report.

## Set up the script that will run the Crew
* This is where we provide the specific topic we want our crew to research.

In [ ]:
#!/usr/bin/env python
# src/research_crew/main.py
import os
from research_crew.crew import ResearchCrew

# Create output directory if it doesn't exist
os.makedirs('output', exist_ok=True)

def run():
    """
    Run the research crew.
    """
    inputs = {
        'topic': 'Artificial Intelligence in Healthcare'
    }

    # Create and run the crew
    result = ResearchCrew().crew().kickoff(inputs=inputs)

    # Print the result
    print("\n\n=== FINAL REPORT ===\n\n")
    print(result.raw)

    print("\n\nReport has been saved to output/report.md")

if __name__ == "__main__":
    run()

## Let's explain the previous code line by line

```python
#!/usr/bin/env python
```
**Shebang line**:  
- Tells the system to **run this script using Python**.
- Useful if you run the script directly from the terminal like: `./main.py`.

---

```python
# src/research_crew/main.py
```
**Comment**:  
- The file is located at `src/research_crew/main.py`.

---

```python
import os
```
**Import**:  
- Loads Python’s built-in `os` module.
- Used for interacting with the operating system (e.g., creating folders).

---

```python
from research_crew.crew import ResearchCrew
```
**Import**:  
- Loads the `ResearchCrew` class we defined earlier from `research_crew/crew.py`.

---

```python
# Create output directory if it doesn't exist
os.makedirs('output', exist_ok=True)
```
**Creates an "output" folder**:  
- If the folder `output/` doesn't already exist, it will **create it**.
- `exist_ok=True` tells Python **not to throw an error** if the folder already exists.

---

```python
def run():
```
**Defines a function** called `run()`.  
- This function will execute the main steps of the program.

---

```python
    """
    Run the research crew.
    """
```
**Docstring**:  
- A short description of what `run()` does.

---

```python
    inputs = {
        'topic': 'Artificial Intelligence in Healthcare'
    }
```
**Create the inputs**:  
- Defines a dictionary called `inputs`.
- It provides the starting **topic** for the research crew: `"Artificial Intelligence in Healthcare"`.

---

```python
    # Create and run the crew
    result = ResearchCrew().crew().kickoff(inputs=inputs)
```
**Main action**:
- `ResearchCrew()` creates an instance of your crew setup.
- `.crew()` gets the assembled Crew object (with agents and tasks).
- `.kickoff(inputs=inputs)` **starts** the process, feeding it the `inputs`.
- **`result`** will hold the **final output** (like the research report).

---

```python
    # Print the result
    print("\n\n=== FINAL REPORT ===\n\n")
```
**Prints a nice header** to the console before showing the report.

---

```python
    print(result.raw)
```
**Prints the raw final report** text that the agents created.

---

```python
    print("\n\nReport has been saved to output/report.md")
```
**Prints a final message** telling the user that the report was saved into `output/report.md` (remember: the Analyst Task saves it there).

---

```python
if __name__ == "__main__":
    run()
```
**Run the script**:
- This standard Python pattern means:  
  > "If this file is executed directly (not imported), call the `run()` function."
- It starts the whole process.

---

#### Quick Overall Summary:
- Create an output folder if needed.
- Enter the topic.
- Create the crew.
- Start the work.
- Print and save the final report.

## Step 7: Make sure you have the environment variables in the .env file
* Make sure you have created a .env file in your project root with your API keys:

In [ ]:
OPENAI_API_KEY=your_openai_api_key
SERPER_API_KEY=your_serper_api_key

* Remember that you can get a Serper API key from [Serper.dev](https://serper.dev/)

## Step 8: Install Dependencies
From terminal:
* `crewai install`

This command will:
* Read the dependencies from your project configuration.
* Create a virtual environment if needed.
* Install all required packages.

## Step 9: Run the Crew
From terminal:
* `crewai run`

When you run this command, you’ll see your crew in action:
* The researcher will gather information about the specified topic,
* and the analyst will then create a comprehensive report based on that research.

You’ll see the agents’ thought processes, actions, and outputs in real-time as they work together to complete their tasks.

## Step 10: Review the Output
Once the crew completes its work, you’ll find the final report in the output/report.md file. The report will include:
* An executive summary.
* Detailed information about the topic.
* Analysis and insights.
* Recommendations or future considerations.